Example of animated circle plots every hour

In [21]:
def save_route_map(route_map, file_name):
    print('CREATE:', file_name)
    route_map.save(file_name)

In [22]:
#https://nbviewer.org/github/python-visualization/folium/blob/main/examples/Plugins.ipynb#Timestamped-GeoJSON
from folium import plugins
import folium

circlemarkers = [{"time": "2017-10-03 01:00", 'radius': 5, 'tooltip': 'hi world', "location": [54.677851,25.282656][::-1]},
                 {"time": "2017-10-03 02:00", 'radius': 5, 'tooltip': 'hi world again', "location": [54.6778603,25.2553013][::-1]},
                 {"time": "2017-10-03 03:00", 'radius': 5, 'tooltip': 'hi world aa', "location": [54.6940226,25.2760565][::-1]},
                 {"time": "2017-10-03 04:00", 'radius': 5, 'tooltip': 'hi world kk', "location": [54.73978779999999,25.2746094][::-1]}]

features = [
    {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": point["location"],
        },
        "properties": {
            "time": point["time"],
            "tooltip": point["tooltip"],
            "id": f"house{i}",
            "icon": "circle",
            "iconstyle": {
                "fillColor": "green",
                "fillOpacity": 0.6,
                "stroke": "true",
                "radius": 7}
        },
    }
    for i, point in enumerate(circlemarkers)
]


m = folium.Map(
    location=[51.467697, -2.548828],
    tiles='openstreetmap',
    zoom_start=16,
)


plugins.TimestampedGeoJson(
    {"type": "FeatureCollection", "features": features},
    period="PT1H",
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options="hh:mm",
    time_slider_drag_update=True,
    duration="PT0H",
).add_to(m)

plugins.LocateControl(auto_start=True).add_to(m)
m
#the render of straight notebook via nbviewer is terrible on phone

In [23]:
save_route_map(m, 'test.html')

CREATE: test.html
